In [0]:
dbutils.fs.ls("/FileStore/tables/")

Out[16]: [FileInfo(path='dbfs:/FileStore/tables/DF_PYSPARk_BDTT.ipynb', name='DF_PYSPARk_BDTT.ipynb', size=33691, modificationTime=1651069762000),
 FileInfo(path='dbfs:/FileStore/tables/Hive_PYSPARk_BDTT.sql', name='Hive_PYSPARk_BDTT.sql', size=2298, modificationTime=1651069762000),
 FileInfo(path='dbfs:/FileStore/tables/RDD_Pyspark_BDTT-1.ipynb', name='RDD_Pyspark_BDTT-1.ipynb', size=57413, modificationTime=1651069763000),
 FileInfo(path='dbfs:/FileStore/tables/RDD_Pyspark_BDTT.ipynb', name='RDD_Pyspark_BDTT.ipynb', size=57413, modificationTime=1651069692000),
 FileInfo(path='dbfs:/FileStore/tables/accounts-1.zip', name='accounts-1.zip', size=5297592, modificationTime=1647956234000),
 FileInfo(path='dbfs:/FileStore/tables/accounts.zip', name='accounts.zip', size=5297592, modificationTime=1645027718000),
 FileInfo(path='dbfs:/FileStore/tables/activations/', name='activations/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/activations.zip/', name='activations.zip/

In [0]:
dbutils.fs.cp("dbfs:/FileStore/tables/clinicaltrial_2021_csv.gz","file:/tmp")

Out[17]: True

In [0]:
%sh
gunzip /tmp/clinicaltrial_2021_csv.gz

In [0]:
%sh
ls /tmp/

Rserv
Rtmp8Bxe7h
chauffeur-daemon-params
chauffeur-daemon.pid
chauffeur-env.sh
clinicaltrial_2021_csv
custom-spark.conf
driver-daemon-params
driver-daemon.pid
driver-env.sh
hsperfdata_root
ipykernel-connection-ReplId-10c40-e6668-641fa-a.json
ipykernel-connection-ReplId-66589-29ace-b2e28-f.json
systemd-private-b80d0d7a3e21478291eba098293ff1f5-apache2.service-pKCT0g
systemd-private-b80d0d7a3e21478291eba098293ff1f5-ntp.service-6HibYf
systemd-private-b80d0d7a3e21478291eba098293ff1f5-systemd-logind.service-tlCJuf
systemd-private-b80d0d7a3e21478291eba098293ff1f5-systemd-resolved.service-SrfYsf
tmp.gyd813l7n8


In [0]:
dbutils.fs.mkdirs("dbfs:/FileStore/tables/clinicaltrial_2021")

Out[20]: True

In [0]:
dbutils.fs.mv("file:/tmp/clinicaltrial_2021_csv","dbfs:/FileStore/tables/clinicaltrial_2021.csv")

Out[21]: True

In [0]:
dbutils.fs.ls("/FileStore/tables")

Out[22]: [FileInfo(path='dbfs:/FileStore/tables/DF_PYSPARk_BDTT.ipynb', name='DF_PYSPARk_BDTT.ipynb', size=33691, modificationTime=1651069762000),
 FileInfo(path='dbfs:/FileStore/tables/Hive_PYSPARk_BDTT.sql', name='Hive_PYSPARk_BDTT.sql', size=2298, modificationTime=1651069762000),
 FileInfo(path='dbfs:/FileStore/tables/RDD_Pyspark_BDTT-1.ipynb', name='RDD_Pyspark_BDTT-1.ipynb', size=57413, modificationTime=1651069763000),
 FileInfo(path='dbfs:/FileStore/tables/RDD_Pyspark_BDTT.ipynb', name='RDD_Pyspark_BDTT.ipynb', size=57413, modificationTime=1651069692000),
 FileInfo(path='dbfs:/FileStore/tables/accounts-1.zip', name='accounts-1.zip', size=5297592, modificationTime=1647956234000),
 FileInfo(path='dbfs:/FileStore/tables/accounts.zip', name='accounts.zip', size=5297592, modificationTime=1645027718000),
 FileInfo(path='dbfs:/FileStore/tables/activations/', name='activations/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/activations.zip/', name='activations.zip/

In [0]:
import pyspark.sql.functions as f

trials = spark.read.options(delimiter = "|", header = True, inferSchema = True).csv("/FileStore/tables/clinicaltrial_2021.csv")

pharm = spark.read.options(escape = "\"", header = True, inferSchema = True).csv("/FileStore/tables/pharma.csv")

mesh = spark.read.options(escape = "\"", header = True, inferSchema = True).csv("/FileStore/tables/mesh.csv").withColumn("code", f.split(f.col("tree"), "\.")[0]).withColumn("term", f.trim("term"))

In [0]:
##Problem 1
trials.select('Id').distinct().count()

Out[24]: 387261

In [0]:
##Problem 2
trials.dropna(subset = "Type").groupBy("Type").count().orderBy("count", ascending = False).show()

+--------------------+------+
|                Type| count|
+--------------------+------+
|      Interventional|301472|
|       Observational| 77540|
|Observational [Pa...|  8180|
|     Expanded Access|    69|
+--------------------+------+



In [0]:
##Problem 3
conds = trials.select('Conditions').dropna(subset = "Conditions").withColumn("temp cond", f.explode(f.split(f.col("Conditions"), ","))).groupBy(f.trim("temp cond").alias("Conditions")).count().orderBy("count", ascending = False)

conds.show(5)

+-----------------+-----+
|       Conditions|count|
+-----------------+-----+
|        Carcinoma|13389|
|Diabetes Mellitus|11080|
|        Neoplasms| 9371|
| Breast Neoplasms| 8640|
|         Syndrome| 8032|
+-----------------+-----+
only showing top 5 rows



In [0]:
##Problem 4 
conds.join(mesh, f.col("Conditions") == f.col("term"), "left").groupBy("code").sum("count").orderBy("sum(count)", ascending = False).show(5)

+----+----------+
|code|sum(count)|
+----+----------+
| C04|    143994|
| C23|    136079|
| C01|    106674|
| C14|     94523|
| C10|     92310|
+----+----------+
only showing top 5 rows



In [0]:
##Problem 5
currentpharm = [row['Parent_Company'] for row in pharm.select("Parent_Company").distinct().collect()]

trials.filter(~f.col('Sponsor').isin(currentpharm)).groupBy(f.col("Sponsor")).count().orderBy("count", ascending = False).show(10)

+--------------------+-----+
|             Sponsor|count|
+--------------------+-----+
|National Cancer I...| 3218|
|M.D. Anderson Can...| 2414|
|Assistance Publiq...| 2369|
|         Mayo Clinic| 2300|
|Merck Sharp & Doh...| 2243|
|   Assiut University| 2154|
|Novartis Pharmace...| 2088|
|Massachusetts Gen...| 1971|
|    Cairo University| 1928|
|   Hoffmann-La Roche| 1828|
+--------------------+-----+
only showing top 10 rows



In [0]:
##Problem 6
trials.filter((f.col("Status") == 'Completed') & (f.split(f.col("Completion"), " ")[1] == '2021')).groupBy(f.col("Completion")).count().orderBy("count", ascending = False).show()

+----------+-----+
|Completion|count|
+----------+-----+
|  Mar 2021| 1227|
|  Jan 2021| 1131|
|  Jun 2021| 1094|
|  May 2021|  984|
|  Apr 2021|  967|
|  Feb 2021|  934|
|  Jul 2021|  819|
|  Aug 2021|  700|
|  Sep 2021|  528|
|  Oct 2021|  187|
+----------+-----+

